## imports :  ]

In [75]:
import json

import pandas as pd
import numpy as np

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import re

## Prizepicks Table

### Join Prizepicks data into one table

In [76]:
with open('prizepicks_json.json') as f:
    jsonobj = json.load(f)

count = 0
data_attributes = []

for item in jsonobj['data']:
    example = jsonobj['data'][count]
    data_attributes.append(example)
    count += 1

df_attr = pd.json_normalize(data_attributes)
df_attr = df_attr.rename(columns={'relationships.new_player.data.id': 'Player Name'})

count2 = 0
data_included = []

for item in jsonobj['included']:
    example2 = jsonobj['included'][count2]
    data_included.append(example2)
    count2 += 1

df_included = pd.json_normalize(data_included)

df_included = df_included.rename(columns={'id': 'Player Name'})

In [77]:
inner_join = pd.merge(df_attr, df_included, on='Player Name', how='inner')

### Format Prizepicks table

In [78]:
PRIZEPICKS_table = inner_join[['attributes.name', 'attributes.team', 'attributes.league', 'attributes.stat_type', 'attributes.line_score', 'attributes.updated_at']]
PRIZEPICKS_table.columns = ['Player', 'Team', 'Sport', 'Stat Type', 'Line', 'Last Updated']
PRIZEPICKS_table['Last Updated'] = pd.to_datetime(PRIZEPICKS_table['Last Updated'])
PRIZEPICKS_table['Last Updated'] = PRIZEPICKS_table['Last Updated'].dt.strftime('%m/%d/%y %I:%M %p')

C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\4190268736.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PRIZEPICKS_table['Last Updated'] = pd.to_datetime(PRIZEPICKS_table['Last Updated'])
C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\4190268736.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PRIZEPICKS_table['Last Updated'] = PRIZEPICKS_table['Last Updated'].dt.strftime('%m/%d/%y %I:%M %p')


Find similiar bets from Draftkings and Prizepicks

## MLB

In [79]:
# MLB Hits Allowed
with open('draftkings_MLB_HA.txt', 'r') as file:
    HitsAllowed = file.read()
HitsAllowed = HitsAllowed.replace("âˆ’", "-")

HA_Line = re.findall('line">(\d+.\d+)', str(HitsAllowed))

HA_Odds = re.findall('color">(.\d+)', str(HitsAllowed))

HA_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(HitsAllowed))


# MLB Strikeouts
with open('draftkings_MLB_SO.txt', 'r') as file:
    Strikeouts = file.read()
Strikeouts = Strikeouts.replace("âˆ’", "-")

SO_Line = re.findall('line">(\d+.\d+)', str(Strikeouts))

SO_Odds = re.findall('color">(.\d+)', str(Strikeouts))

SO_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(Strikeouts))

# MLB Strikeouts
with open('draftkings_MLB_Runs_Scored.txt', 'r') as file:
    Runs = file.read()
Runs = Runs.replace("âˆ’", "-")

Runs_Line = re.findall('line">(\d+.\d+)', str(Runs))

Runs_Odds = re.findall('color">(.\d+)', str(Runs))

Runs_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(Runs))

In [80]:
Upd_Odds = []
Upd_Lines = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in HA_Line:
    Upd_Lines.append(HA_Line[First:Second])
    First += 2
    Second += 2

for i in HA_Odds:
    Upd_Odds.append(HA_Odds[First2:Second2])
    First2 += 2
    Second2 += 2


HA_Odds_DF = pd.DataFrame(Upd_Odds)
HA_Odds_DF = HA_Odds_DF.dropna()

HA_Lines_DF = pd.DataFrame(Upd_Lines)
HA_Lines_DF = HA_Lines_DF.dropna()
HA_Lines_DF = HA_Lines_DF.drop([0],axis=1)

In [81]:
Upd_Odds2 = []
Upd_Lines2 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in SO_Line:
    Upd_Lines2.append(SO_Line[First:Second])
    First += 2
    Second += 2

for i in SO_Odds:
    Upd_Odds2.append(SO_Odds[First2:Second2])
    First2 += 2
    Second2 += 2

SO_Odds_DF = pd.DataFrame(Upd_Odds2)
SO_Odds_DF = SO_Odds_DF.dropna()

SO_Lines_DF = pd.DataFrame(Upd_Lines2)
SO_Lines_DF = SO_Lines_DF.dropna()
SO_Lines_DF = SO_Lines_DF.drop([0],axis=1)

In [82]:
Upd_Odds8 = []
Upd_Lines8 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in Runs_Line:
    Upd_Lines8.append(Runs_Line[First:Second])
    First += 2
    Second += 2

for i in Runs_Odds:
    Upd_Odds8.append(Runs_Odds[First2:Second2])
    First2 += 2
    Second2 += 2

Runs_Odds_DF = pd.DataFrame(Upd_Odds8)
Runs_Odds_DF = Runs_Odds_DF.dropna()

Runs_Lines_DF = pd.DataFrame(Upd_Lines8)
Runs_Lines_DF = Runs_Lines_DF.dropna()
Runs_Lines_DF = Runs_Lines_DF.drop([0],axis=1)

### Create dataframe for MLB stats

In [83]:
HITSALLOWED_DK = pd.DataFrame(list(zip(HA_Players)),
               columns =['Name'])

HITSALLOWED_DK = HITSALLOWED_DK.join(HA_Lines_DF)
HITSALLOWED_DK = HITSALLOWED_DK.join(HA_Odds_DF, lsuffix="fa", rsuffix="werf")

HITSALLOWED_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

HITSALLOWED_DK['O Odds'] = pd.to_numeric(HITSALLOWED_DK['O Odds'])
HITSALLOWED_DK['U Odds'] = pd.to_numeric(HITSALLOWED_DK['U Odds'])
HITSALLOWED_DK['Line'] = pd.to_numeric(HITSALLOWED_DK['Line'])
HITSALLOWED_DK.head()

,Player,Line,O Odds,U Odds
0,Jameson Taillon,5.5,125,-170
1,Jon Gray,3.5,-170,125
2,Cal Quantrill,4.5,-165,120
3,Daniel Lynch,5.5,-105,-125
4,Chris Flexen,4.5,-115,-115


In [84]:
STRIKEOUTS_DK = pd.DataFrame(list(zip(SO_Players)),
               columns =['Name'])

STRIKEOUTS_DK = STRIKEOUTS_DK.join(SO_Lines_DF)
STRIKEOUTS_DK = STRIKEOUTS_DK.join(SO_Odds_DF, lsuffix="fa", rsuffix="werf")

STRIKEOUTS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

STRIKEOUTS_DK['O Odds'] = pd.to_numeric(STRIKEOUTS_DK['O Odds'])
STRIKEOUTS_DK['U Odds'] = pd.to_numeric(STRIKEOUTS_DK['U Odds'])
STRIKEOUTS_DK['Line'] = pd.to_numeric(STRIKEOUTS_DK['Line'])
STRIKEOUTS_DK.head()

,Player,Line,O Odds,U Odds
0,Jameson Taillon,4.5,-140,105
1,Jon Gray,5.5,-115,-115
2,Cal Quantrill,4.5,105,-135
3,Daniel Lynch,4.5,115,-150
4,Chris Flexen,3.5,-130,-105


In [85]:
RUNS_DK = pd.DataFrame(list(zip(Runs_Players)),
               columns =['Name'])

RUNS_DK = RUNS_DK.join(Runs_Lines_DF)
RUNS_DK = RUNS_DK.join(Runs_Odds_DF, lsuffix="fa", rsuffix="werf")

RUNS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

RUNS_DK['O Odds'] = pd.to_numeric(RUNS_DK['O Odds'])
RUNS_DK['U Odds'] = pd.to_numeric(RUNS_DK['U Odds'])
RUNS_DK['Line'] = pd.to_numeric(RUNS_DK['Line'])
RUNS_DK.head()

,Player,Line,O Odds,U Odds
0,Aaron Hicks,0.5,170,-230
1,Aaron Judge,0.5,-135,105
2,Adolis Garcia,0.5,150,-200
3,Anthony Rizzo,0.5,135,-180
4,Bubba Thompson,0.5,190,-265


#### Format <b>DK</b> (MLB-SO) dataframe

In [86]:
STRIKEOUTS_DK['O Prob'] = (STRIKEOUTS_DK['O Odds'].abs()/(STRIKEOUTS_DK['O Odds'].abs()+100)*100).round(1).where(STRIKEOUTS_DK['O Odds'] < 0, 100/(STRIKEOUTS_DK['O Odds']+100)*100).round(1)
STRIKEOUTS_DK['U Prob'] = (STRIKEOUTS_DK['U Odds'].abs()/(STRIKEOUTS_DK['U Odds'].abs()+100)*100).round(1).where(STRIKEOUTS_DK['U Odds'] < 0, 100/(STRIKEOUTS_DK['U Odds']+100)*100).round(1)
STRIKEOUTS_DK['Favor'] = np.where(STRIKEOUTS_DK['O Prob']> STRIKEOUTS_DK['U Prob'], "Over", "Under")

STRIKEOUTS_DK = STRIKEOUTS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
STRIKEOUTS_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Jameson Taillon,4.5,58.3,48.8,Over
1,Jon Gray,5.5,53.5,53.5,Under
2,Cal Quantrill,4.5,48.8,57.4,Under
3,Daniel Lynch,4.5,46.5,60.0,Under
4,Chris Flexen,3.5,56.5,51.2,Over


In [87]:
HITSALLOWED_DK['O Prob'] = (HITSALLOWED_DK['O Odds'].abs()/(HITSALLOWED_DK['O Odds'].abs()+100)*100).round(1).where(HITSALLOWED_DK['O Odds'] < 0, 100/(HITSALLOWED_DK['O Odds']+100)*100).round(1)
HITSALLOWED_DK['U Prob'] = (HITSALLOWED_DK['U Odds'].abs()/(HITSALLOWED_DK['U Odds'].abs()+100)*100).round(1).where(HITSALLOWED_DK['U Odds'] < 0, 100/(HITSALLOWED_DK['U Odds']+100)*100).round(1)
HITSALLOWED_DK['Favor'] = np.where(HITSALLOWED_DK['O Prob']> HITSALLOWED_DK['U Prob'], "Over", "Under")

HITSALLOWED_DK = HITSALLOWED_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
HITSALLOWED_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Jameson Taillon,5.5,44.4,63.0,Under
1,Jon Gray,3.5,63.0,44.4,Over
2,Cal Quantrill,4.5,62.3,45.5,Over
3,Daniel Lynch,5.5,51.2,55.6,Under
4,Chris Flexen,4.5,53.5,53.5,Under


In [88]:
RUNS_DK['O Prob'] = (RUNS_DK['O Odds'].abs()/(RUNS_DK['O Odds'].abs()+100)*100).round(1).where(RUNS_DK['O Odds'] < 0, 100/(RUNS_DK['O Odds']+100)*100).round(1)
RUNS_DK['U Prob'] = (RUNS_DK['U Odds'].abs()/(RUNS_DK['U Odds'].abs()+100)*100).round(1).where(RUNS_DK['U Odds'] < 0, 100/(RUNS_DK['U Odds']+100)*100).round(1)
RUNS_DK['Favor'] = np.where(RUNS_DK['O Prob']> RUNS_DK['U Prob'], "Over", "Under")

RUNS_DK = RUNS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
RUNS_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Aaron Hicks,0.5,37.0,69.7,Under
1,Aaron Judge,0.5,57.4,48.8,Over
2,Adolis Garcia,0.5,40.0,66.7,Under
3,Anthony Rizzo,0.5,42.6,64.3,Under
4,Bubba Thompson,0.5,34.5,72.6,Under


In [89]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


STRIKEOUTS_TABLE = fuzzy_merge(PRIZEPICKS_table, STRIKEOUTS_DK, 'Player', 'Player', threshold=95)

STRIKEOUTS_TABLE = STRIKEOUTS_TABLE[STRIKEOUTS_TABLE['Stat Type'].str.contains('Strikeouts')]
STRIKEOUTS_TABLE = STRIKEOUTS_TABLE[STRIKEOUTS_TABLE['Sport'].str.contains('MLB')]
STRIKEOUTS_TABLE['Player'] = STRIKEOUTS_TABLE['matches']

STRIKEOUTS_TABLE = STRIKEOUTS_TABLE.merge(STRIKEOUTS_DK, on='Player')

STRIKEOUTS_TABLE['Line_x'] = np.where(STRIKEOUTS_TABLE['Line_x']!= STRIKEOUTS_TABLE['Line_y'], pd.NaT, "Under")
STRIKEOUTS_TABLE['Player'].replace('', np.nan, inplace=True)
STRIKEOUTS_TABLE = STRIKEOUTS_TABLE.dropna()

STRIKEOUTS_TABLE['Probability (%)'] = np.where(STRIKEOUTS_TABLE['O Prob']== STRIKEOUTS_TABLE['U Prob'], 50, STRIKEOUTS_TABLE[["O Prob", "U Prob"]].max(axis=1))

STRIKEOUTS_TABLE = STRIKEOUTS_TABLE.drop(columns=['matches', 'Line_x', 'O Prob', 'U Prob'], axis=1)
STRIKEOUTS_TABLE = STRIKEOUTS_TABLE.rename(columns={'Line_y': 'Line'})
STRIKEOUTS_TABLE = STRIKEOUTS_TABLE[['Player', 'Team', 'Sport', 'Stat Type', 'Last Updated', 'Line', 'Probability (%)', 'Favor']]

STRIKEOUTS_TABLE.head()

C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\3138489536.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\3138489536.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


,Player,Team,Sport,Stat Type,Last Updated,Line,Probability (%),Favor
13,JT Brubaker,PIT,MLB,Strikeouts,10/04/22 10:35 AM,3.5,56.5,Under
16,Braxton Garrett,MIA,MLB,Strikeouts,10/04/22 10:36 AM,4.5,60.8,Under
26,Zac Gallen,ARI,MLB,Strikeouts,10/04/22 10:35 AM,6.5,56.5,Over
40,Michael Lorenzen,LAA,MLB,Strikeouts,10/04/22 10:37 AM,4.5,55.6,Under


In [90]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


HITSALLOWED_TABLE = fuzzy_merge(PRIZEPICKS_table, HITSALLOWED_DK, 'Player', 'Player', threshold=95)

HITSALLOWED_TABLE = HITSALLOWED_TABLE[HITSALLOWED_TABLE['Stat Type'].str.contains('Hits Allowed')]
HITSALLOWED_TABLE = HITSALLOWED_TABLE[HITSALLOWED_TABLE['Sport'].str.contains('MLB')]
HITSALLOWED_TABLE['Player'] = HITSALLOWED_TABLE['matches']

HITSALLOWED_TABLE = HITSALLOWED_TABLE.merge(HITSALLOWED_DK, on='Player')

HITSALLOWED_TABLE['Line_x'] = np.where(HITSALLOWED_TABLE['Line_x']!= HITSALLOWED_TABLE['Line_y'], pd.NaT, "Under")
HITSALLOWED_TABLE['Player'].replace('', np.nan, inplace=True)
HITSALLOWED_TABLE = HITSALLOWED_TABLE.dropna()

HITSALLOWED_TABLE['Probability (%)'] = np.where(HITSALLOWED_TABLE['O Prob']== HITSALLOWED_TABLE['U Prob'], 50, HITSALLOWED_TABLE[["O Prob", "U Prob"]].max(axis=1))

HITSALLOWED_TABLE = HITSALLOWED_TABLE.drop(columns=['matches', 'Line_x', 'O Prob', 'U Prob'], axis=1)
HITSALLOWED_TABLE = HITSALLOWED_TABLE.rename(columns={'Line_y': 'Line'})
HITSALLOWED_TABLE = HITSALLOWED_TABLE[['Player', 'Team', 'Sport', 'Stat Type', 'Last Updated', 'Line', 'Probability (%)', 'Favor']]

HITSALLOWED_TABLE.head()

C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\2621633047.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\2621633047.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


,Player,Team,Sport,Stat Type,Last Updated,Line,Probability (%),Favor
2,Eduardo Rodriguez,DET,MLB,Hits Allowed,10/04/22 10:49 AM,5.5,56.5,Under
3,Chris Flexen,SEA,MLB,Hits Allowed,10/04/22 10:43 AM,4.5,50.0,Under
4,Jake Odorizzi,ATL,MLB,Hits Allowed,10/04/22 10:42 AM,4.5,57.4,Over
5,Braxton Garrett,MIA,MLB,Hits Allowed,10/04/22 10:44 AM,4.5,58.3,Under
6,Javier Assad,CHC,MLB,Hits Allowed,10/04/22 10:39 AM,4.5,59.2,Under


In [91]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


RUNS_TABLE = fuzzy_merge(PRIZEPICKS_table, RUNS_DK, 'Player', 'Player', threshold=95)

RUNS_TABLE = RUNS_TABLE[RUNS_TABLE['Stat Type'].str.endswith('Runs')]
RUNS_TABLE = RUNS_TABLE[RUNS_TABLE['Sport'].str.contains('MLB')]
RUNS_TABLE['Player'] = RUNS_TABLE['matches']

RUNS_TABLE = RUNS_TABLE.merge(RUNS_DK, on='Player')

RUNS_TABLE['Line_x'] = np.where(RUNS_TABLE['Line_x']!= RUNS_TABLE['Line_y'], pd.NaT, "Under")
RUNS_TABLE['Player'].replace('', np.nan, inplace=True)
RUNS_TABLE = RUNS_TABLE.dropna()

RUNS_TABLE['Probability (%)'] = np.where(RUNS_TABLE['O Prob']== RUNS_TABLE['U Prob'], 50, RUNS_TABLE[["O Prob", "U Prob"]].max(axis=1))

RUNS_TABLE = RUNS_TABLE.drop(columns=['matches', 'Line_x', 'O Prob', 'U Prob'], axis=1)
RUNS_TABLE = RUNS_TABLE.rename(columns={'Line_y': 'Line'})
RUNS_TABLE = RUNS_TABLE[['Player', 'Team', 'Sport', 'Stat Type', 'Last Updated', 'Line', 'Probability (%)', 'Favor']]

RUNS_TABLE.head()

C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\3279307682.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\3279307682.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


,Player,Team,Sport,Stat Type,Last Updated,Line,Probability (%),Favor
0,Bo Bichette,TOR,MLB,Runs,10/04/22 12:07 PM,0.5,54.5,Under
1,George Springer,TOR,MLB,Runs,10/04/22 12:07 PM,0.5,56.5,Over
2,Freddie Freeman,LAD,MLB,Runs,10/04/22 12:09 PM,0.5,54.5,Under
3,Max Muncy,LAD,MLB,Runs,10/04/22 12:25 PM,0.5,54.5,Under
4,Mookie Betts,LAD,MLB,Runs,10/04/22 12:09 PM,0.5,55.6,Under


### Join Prizepicks data into one table

In [92]:
with open('prizepicks_json.json') as f:
    jsonobj = json.load(f)

count = 0
data_attributes = []

for item in jsonobj['data']:
    example = jsonobj['data'][count]
    data_attributes.append(example)
    count += 1

df_attr = pd.json_normalize(data_attributes)
df_attr = df_attr.rename(columns={'relationships.new_player.data.id': 'Player Name'})

count2 = 0
data_included = []

for item in jsonobj['included']:
    example2 = jsonobj['included'][count2]
    data_included.append(example2)
    count2 += 1

df_included = pd.json_normalize(data_included)

df_included = df_included.rename(columns={'id': 'Player Name'})

In [93]:
inner_join = pd.merge(df_attr, df_included, on='Player Name', how='inner')

### Format Prizepicks table

In [94]:
PRIZEPICKS_table = inner_join[['attributes.name', 'attributes.team', 'attributes.league', 'attributes.stat_type', 'attributes.line_score', 'attributes.updated_at']]
PRIZEPICKS_table.columns = ['Player', 'Team', 'Sport', 'Stat Type', 'Line', 'Last Updated']
PRIZEPICKS_table['Last Updated'] = pd.to_datetime(PRIZEPICKS_table['Last Updated'])
PRIZEPICKS_table['Last Updated'] = PRIZEPICKS_table['Last Updated'].dt.strftime('%m/%d/%y %I:%M %p')

C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\4190268736.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PRIZEPICKS_table['Last Updated'] = pd.to_datetime(PRIZEPICKS_table['Last Updated'])
C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\4190268736.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PRIZEPICKS_table['Last Updated'] = PRIZEPICKS_table['Last Updated'].dt.strftime('%m/%d/%y %I:%M %p')


Find similiar bets from Draftkings and Prizepicks

## NFL

### Open and do some cleaning

In [95]:
# NFL Passing Yards
with open('draftkings_NFL_PassYds.txt', 'r') as file:
    PassYds = file.read()
PassYds = PassYds.replace("âˆ’", "-")

PassYds_Line = re.findall('line">(\d+.\d+)', str(PassYds))

PassYds_Odds = re.findall('color">(.\d+)', str(PassYds))

PassYds_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(PassYds))


# NFL Passing Touchdowns
with open('draftkings_NFL_PassTds.txt', 'r') as file:
    PassTds = file.read()
PassTds = PassTds.replace("âˆ’", "-")

PassTds_Line = re.findall('line">(\d+.\d+)', str(PassTds))

PassTds_Odds = re.findall('color">(.\d+)', str(PassTds))

PassTds_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(PassTds))

# NFL Rush Yards
with open('draftkings_NFL_RushYds.txt', 'r') as file:
    RushYds = file.read()
RushYds = RushYds.replace("âˆ’", "-")

RushYds_Line = re.findall('line">(\d+.\d+)', str(RushYds))

RushYds_Odds = re.findall('color">(.\d+)', str(RushYds))

RushYds_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(RushYds))

### Make dataframes from data

In [96]:
Upd_Odds3 = []
Upd_Lines3 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in PassYds_Line:
    Upd_Lines3.append(PassYds_Line[First:Second])
    First += 2
    Second += 2

for i in PassYds_Odds:
    Upd_Odds3.append(PassYds_Odds[First2:Second2])
    First2 += 2
    Second2 += 2


PassYds_Odds_DF = pd.DataFrame(Upd_Odds3)
PassYds_Odds_DF = PassYds_Odds_DF.dropna()

PassYds_Lines_DF = pd.DataFrame(Upd_Lines3)
PassYds_Lines_DF = PassYds_Lines_DF.dropna()
PassYds_Lines_DF = PassYds_Lines_DF.drop([0],axis=1)

In [97]:
Upd_Odds4 = []
Upd_Lines4 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in PassTds_Line:
    Upd_Lines4.append(PassTds_Line[First:Second])
    First += 2
    Second += 2

for i in PassTds_Odds:
    Upd_Odds4.append(PassTds_Odds[First2:Second2])
    First2 += 2
    Second2 += 2

PassTds_Odds_DF = pd.DataFrame(Upd_Odds4)
PassTds_Odds_DF = PassTds_Odds_DF.dropna()

PassTds_Lines_DF = pd.DataFrame(Upd_Lines4)
PassTds_Lines_DF = PassTds_Lines_DF.dropna()
PassTds_Lines_DF = PassTds_Lines_DF.drop([0],axis=1)


In [98]:
Upd_Odds5 = []
Upd_Lines5 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in RushYds_Line:
    Upd_Lines5.append(RushYds_Line[First:Second])
    First += 2
    Second += 2

for i in RushYds_Odds:
    Upd_Odds5.append(RushYds_Odds[First2:Second2])
    First2 += 2
    Second2 += 2

RushYds_Odds_DF = pd.DataFrame(Upd_Odds5)
RushYds_Odds_DF = RushYds_Odds_DF.dropna()

RushYds_Lines_DF = pd.DataFrame(Upd_Lines5)
RushYds_Lines_DF = RushYds_Lines_DF.dropna()
RushYds_Lines_DF = RushYds_Lines_DF.drop([0],axis=1)


In [99]:
PASSYDS_DK = pd.DataFrame(list(zip(PassYds_Players)),
               columns =['Name'])

PASSYDS_DK = PASSYDS_DK.join(PassYds_Lines_DF)
PASSYDS_DK = PASSYDS_DK.join(PassYds_Odds_DF, lsuffix="fa", rsuffix="werf")

PASSYDS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

PASSYDS_DK[['O Odds', 'Line', 'U Odds']] = PASSYDS_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)

In [100]:
PASSTDS_DK = pd.DataFrame(list(zip(PassTds_Players)),
               columns =['Name'])

PASSTDS_DK = PASSTDS_DK.join(PassTds_Lines_DF)
PASSTDS_DK = PASSTDS_DK.join(PassTds_Odds_DF, lsuffix="fa", rsuffix="werf")

PASSTDS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

PASSTDS_DK[['O Odds', 'Line', 'U Odds']] = PASSTDS_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)


In [101]:
RUSHYDS_DK = pd.DataFrame(list(zip(RushYds_Players)),
               columns =['Name'])

RUSHYDS_DK = RUSHYDS_DK.join(RushYds_Lines_DF)
RUSHYDS_DK = RUSHYDS_DK.join(RushYds_Odds_DF, lsuffix="fa", rsuffix="werf")

RUSHYDS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

RUSHYDS_DK[['O Odds', 'Line', 'U Odds']] = RUSHYDS_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)

### Format columns

In [102]:
PASSYDS_DK['O Prob'] = (PASSYDS_DK['O Odds'].abs()/(PASSYDS_DK['O Odds'].abs()+100)*100).round(1).where(PASSYDS_DK['O Odds'] < 0, 100/(PASSYDS_DK['O Odds']+100)*100).round(1)
PASSYDS_DK['U Prob'] = (PASSYDS_DK['U Odds'].abs()/(PASSYDS_DK['U Odds'].abs()+100)*100).round(1).where(PASSYDS_DK['U Odds'] < 0, 100/(PASSYDS_DK['U Odds']+100)*100).round(1)
PASSYDS_DK['Favor'] = np.where(PASSYDS_DK['O Prob']> PASSYDS_DK['U Prob'], "Over", "Under")

PASSYDS_DK = PASSYDS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
PASSYDS_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Matt Ryan,234.5,53.5,53.5,Under
1,Russell Wilson,225.5,57.4,50.0,Over


In [103]:
PASSTDS_DK['O Prob'] = (PASSTDS_DK['O Odds'].abs()/(PASSTDS_DK['O Odds'].abs()+100)*100).round(1).where(PASSTDS_DK['O Odds'] < 0, 100/(PASSTDS_DK['O Odds']+100)*100).round(1)
PASSTDS_DK['U Prob'] = (PASSTDS_DK['U Odds'].abs()/(PASSTDS_DK['U Odds'].abs()+100)*100).round(1).where(PASSTDS_DK['U Odds'] < 0, 100/(PASSTDS_DK['U Odds']+100)*100).round(1)
PASSTDS_DK['Favor'] = np.where(PASSTDS_DK['O Prob']> PASSTDS_DK['U Prob'], "Over", "Under")

PASSTDS_DK = PASSTDS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
PASSTDS_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Matt Ryan,1.5,45.5,61.5,Under
1,Russell Wilson,1.5,46.5,60.0,Under


In [104]:
RUSHYDS_DK['O Prob'] = (RUSHYDS_DK['O Odds'].abs()/(RUSHYDS_DK['O Odds'].abs()+100)*100).round(1).where(RUSHYDS_DK['O Odds'] < 0, 100/(RUSHYDS_DK['O Odds']+100)*100).round(1)
RUSHYDS_DK['U Prob'] = (RUSHYDS_DK['U Odds'].abs()/(RUSHYDS_DK['U Odds'].abs()+100)*100).round(1).where(RUSHYDS_DK['U Odds'] < 0, 100/(RUSHYDS_DK['U Odds']+100)*100).round(1)
RUSHYDS_DK['Favor'] = np.where(RUSHYDS_DK['O Prob']> RUSHYDS_DK['U Prob'], "Over", "Under")

RUSHYDS_DK = RUSHYDS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
RUSHYDS_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Matt Ryan,1.5,55.6,51.2,Over


### Match lines from Prizepicks to Draftkings

In [105]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


PASSYDS_TABLE = fuzzy_merge(PRIZEPICKS_table, PASSYDS_DK, 'Player', 'Player', threshold=95)

PASSYDS_TABLE = PASSYDS_TABLE[PASSYDS_TABLE['Stat Type'].str.contains('Pass Yards')]
PASSYDS_TABLE = PASSYDS_TABLE[PASSYDS_TABLE['Sport'].str.endswith('NFL')]
PASSYDS_TABLE['Player'] = PASSYDS_TABLE['matches']

PASSYDS_TABLE = PASSYDS_TABLE.merge(PASSYDS_DK, on='Player')

PASSYDS_TABLE['Line_x'] = np.where(PASSYDS_TABLE['Line_x']!= PASSYDS_TABLE['Line_y'], pd.NaT, "Under")
PASSYDS_TABLE['Player'].replace('', np.nan, inplace=True)
PASSYDS_TABLE = PASSYDS_TABLE.dropna()

PASSYDS_TABLE['Probability (%)'] = np.where(PASSYDS_TABLE['O Prob']== PASSYDS_TABLE['U Prob'], 50, PASSYDS_TABLE[["O Prob", "U Prob"]].max(axis=1))


PASSYDS_TABLE = PASSYDS_TABLE.drop(columns=['matches', 'Line_x', 'O Prob', 'U Prob'], axis=1)
PASSYDS_TABLE = PASSYDS_TABLE.rename(columns={'Line_y': 'Line'})
PASSYDS_TABLE = PASSYDS_TABLE[['Player', 'Team', 'Sport', 'Stat Type', 'Last Updated', 'Line', 'Probability (%)', 'Favor']]

PASSYDS_TABLE

C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\3728978377.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\3728978377.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


,Player,Team,Sport,Stat Type,Last Updated,Line,Probability (%),Favor
1,Russell Wilson,DEN,NFL,Pass Yards,10/03/22 07:54 PM,225.5,57.4,Over


In [106]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


PASSTDS_TABLE = fuzzy_merge(PRIZEPICKS_table, PASSTDS_DK, 'Player', 'Player', threshold=95)

PASSTDS_TABLE = PASSTDS_TABLE[PASSTDS_TABLE['Stat Type'].str.contains('Pass TDs')]
PASSTDS_TABLE = PASSTDS_TABLE[PASSTDS_TABLE['Sport'].str.endswith('NFL')]
PASSTDS_TABLE['Player'] = PASSTDS_TABLE['matches']

PASSTDS_TABLE = PASSTDS_TABLE.merge(PASSTDS_DK, on='Player')

PASSTDS_TABLE['Line_x'] = np.where(PASSTDS_TABLE['Line_x']!= PASSTDS_TABLE['Line_y'], pd.NaT, "Under")
PASSTDS_TABLE['Player'].replace('', np.nan, inplace=True)
PASSTDS_TABLE = PASSTDS_TABLE.dropna()

PASSTDS_TABLE['Probability (%)'] = np.where(PASSTDS_TABLE['O Prob']== PASSTDS_TABLE['U Prob'], 50, PASSTDS_TABLE[["O Prob", "U Prob"]].max(axis=1))

PASSTDS_TABLE = PASSTDS_TABLE.drop(columns=['matches', 'Line_x', 'O Prob', 'U Prob'], axis=1)
PASSTDS_TABLE = PASSTDS_TABLE.rename(columns={'Line_y': 'Line'})
PASSTDS_TABLE = PASSTDS_TABLE[['Player', 'Team', 'Sport', 'Stat Type', 'Last Updated', 'Line', 'Probability (%)', 'Favor']]

PASSTDS_TABLE

C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\1492721847.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\1492721847.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


,Player,Team,Sport,Stat Type,Last Updated,Line,Probability (%),Favor


In [107]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


RUSHYDS_TABLE = fuzzy_merge(PRIZEPICKS_table, RUSHYDS_DK, 'Player', 'Player', threshold=95)

RUSHYDS_TABLE = RUSHYDS_TABLE[RUSHYDS_TABLE['Stat Type'].str.contains('Rush Yards')]
RUSHYDS_TABLE = RUSHYDS_TABLE[RUSHYDS_TABLE['Sport'].str.endswith('NFL')]
RUSHYDS_TABLE['Player'] = RUSHYDS_TABLE['matches']

RUSHYDS_TABLE = RUSHYDS_TABLE.merge(RUSHYDS_DK, on='Player')

RUSHYDS_TABLE['Line_x'] = np.where(RUSHYDS_TABLE['Line_x']!= RUSHYDS_TABLE['Line_y'], pd.NaT, "Under")
RUSHYDS_TABLE['Player'].replace('', np.nan, inplace=True)
RUSHYDS_TABLE = RUSHYDS_TABLE.dropna()

RUSHYDS_TABLE['Probability (%)'] = np.where(RUSHYDS_TABLE['O Prob']== RUSHYDS_TABLE['U Prob'], 50, RUSHYDS_TABLE[["O Prob", "U Prob"]].max(axis=1))

RUSHYDS_TABLE = RUSHYDS_TABLE.drop(columns=['matches', 'Line_x', 'O Prob', 'U Prob'], axis=1)
RUSHYDS_TABLE = RUSHYDS_TABLE.rename(columns={'Line_y': 'Line'})
RUSHYDS_TABLE = RUSHYDS_TABLE[['Player', 'Team', 'Sport', 'Stat Type', 'Last Updated', 'Line', 'Probability (%)', 'Favor']]

RUSHYDS_TABLE

C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\2740397701.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
C:\Users\Paul\AppData\Local\Temp\ipykernel_9680\2740397701.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


,Player,Team,Sport,Stat Type,Last Updated,Line,Probability (%),Favor
0,Matt Ryan,IND,NFL,Rush Yards,10/04/22 10:13 AM,1.5,55.6,Over


## NBA

In [108]:
# NBA Pts
# with open('draftkings_NBA_PTS.txt', 'r') as file:
#     Pts = file.read()
# Pts = Pts.replace("âˆ’", "-")

# Pts_Line = re.findall('line">(\d+.\d+)', str(Pts))

# Pts_Odds = re.findall('color">(.\d+)', str(Pts))

# Pts_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(Pts))
# print(Pts_Players)

## Main table

In [109]:
MAIN_TABLE = pd.concat([HITSALLOWED_TABLE, STRIKEOUTS_TABLE, PASSYDS_TABLE, PASSTDS_TABLE, RUSHYDS_TABLE, RUNS_TABLE])
MAIN_TABLE.to_csv("MAIN_TABLE.csv", index=False)
print(len(MAIN_TABLE))
MAIN_TABLE

20


,Player,Team,Sport,Stat Type,Last Updated,Line,Probability (%),Favor
2,Eduardo Rodriguez,DET,MLB,Hits Allowed,10/04/22 10:49 AM,5.5,56.5,Under
3,Chris Flexen,SEA,MLB,Hits Allowed,10/04/22 10:43 AM,4.5,50.0,Under
4,Jake Odorizzi,ATL,MLB,Hits Allowed,10/04/22 10:42 AM,4.5,57.4,Over
5,Braxton Garrett,MIA,MLB,Hits Allowed,10/04/22 10:44 AM,4.5,58.3,Under
6,Javier Assad,CHC,MLB,Hits Allowed,10/04/22 10:39 AM,4.5,59.2,Under
8,Mitch White,TOR,MLB,Hits Allowed,10/04/22 10:40 AM,4.5,57.4,Under
9,Jeffrey Springs,TB,MLB,Hits Allowed,10/04/22 11:05 AM,4.5,56.5,Over
14,Justin Verlander,HOU,MLB,Hits Allowed,10/04/22 10:42 AM,4.5,60.8,Over
17,Michael Lorenzen,LAA,MLB,Hits Allowed,10/04/22 10:48 AM,4.5,54.5,Under
13,JT Brubaker,PIT,MLB,Strikeouts,10/04/22 10:35 AM,3.5,56.5,Under


In [110]:
# Importing required library
import pygsheets
  
# Create the Client
# Enter the name of the downloaded KEYS 
# file in service_account_file
client = pygsheets.authorize(service_account_file="main-table-364421-340b276b1a8e.json")


sh = client.open('Main Table')
wks = sh[0]
wks.clear()
wks.set_dataframe(MAIN_TABLE,(1,1))